In [ ]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import TIFUKNN
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:12<00:00, 604.73it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 5557.89it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3054.27it/s]


In [ ]:
def objective(trial):
    params = {
        "model": TIFUKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            group_num=trial.suggest_int("group_num", low=2, high=10, step=1),
            within_decay_rate=trial.suggest_float("within_decay_rate", 0.1, 1.0, step=0.1),
            group_decay_rate=trial.suggest_float("group_decay_rate", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.1),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-23 21:05:19,176] A new study created in memory with name: no-name-a58cd8bc-cd3c-4819-8da4-03b99e67430e


TIFUKNN fitting...


100%|██████████| 7357/7357 [08:34<00:00, 14.30it/s]
[I 2023-04-23 21:14:09,924] Trial 0 finished with value: 0.08107959365618923 and parameters: {'group_num': 8, 'within_decay_rate': 0.1, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 375, 'alpha': 0.5}. Best is trial 0 with value: 0.08107959365618923.


TIFUKNN fitting...


100%|██████████| 7357/7357 [05:28<00:00, 22.40it/s]
[I 2023-04-23 21:19:53,719] Trial 1 finished with value: 0.06082132891301551 and parameters: {'group_num': 4, 'within_decay_rate': 0.2, 'group_decay_rate': 0.8, 'nearest_neighbors_num': 100, 'alpha': 0.0}. Best is trial 0 with value: 0.08107959365618923.


...

100%|██████████| 7357/7357 [09:01<00:00, 13.59it/s]
[I 2023-04-24 00:16:09,582] Trial 22 finished with value: 0.10950544725367108 and parameters: {'group_num': 10, 'within_decay_rate': 0.9, 'group_decay_rate': 1.0, 'nearest_neighbors_num': 425, 'alpha': 0.4}. Best is trial 21 with value: 0.11105743291942718.


TIFUKNN fitting...


100%|██████████| 7357/7357 [08:47<00:00, 13.94it/s]
[I 2023-04-24 00:25:12,944] Trial 23 finished with value: 0.10939951387561178 and parameters: {'group_num': 9, 'within_decay_rate': 0.9, 'group_decay_rate': 1.0, 'nearest_neighbors_num': 400, 'alpha': 0.4}. Best is trial 21 with value: 0.11105743291942718.


TIFUKNN fitting...


100%|██████████| 7357/7357 [09:21<00:00, 13.11it/s]
[I 2023-04-24 00:34:50,604] Trial 24 finished with value: 0.10918993090481564 and parameters: {'group_num': 10, 'within_decay_rate': 1.0, 'group_decay_rate': 0.9, 'nearest_neighbors_num': 450, 'alpha': 0.5}. Best is trial 21 with value: 0.11105743291942718.


Test:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": TIFUKNN(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        group_num=study.best_params["group_num"],
        within_decay_rate=study.best_params["within_decay_rate"],
        group_decay_rate=study.best_params["group_decay_rate"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 7358/7358 [00:11<00:00, 652.68it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 5238.99it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4507.02it/s]


TIFUKNN fitting...


100%|██████████| 7358/7358 [00:08<00:00, 876.22it/s] 


In [ ]:
trainer.evaluate(mode="test")

100%|██████████| 7357/7357 [06:00<00:00, 20.38it/s]


{'precision': 0.057401114584749224,
 'recall': 0.135966844520265,
 'ndcg': 0.11790728194701038}

# TaoBao

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:34<00:00, 289.52it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 24261.11it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 18817.06it/s]


In [ ]:
def objective(trial):
    params = {
        "model": TIFUKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            group_num=trial.suggest_int("group_num", low=2, high=10, step=1),
            within_decay_rate=trial.suggest_float("within_decay_rate", 0.1, 1.0, step=0.1),
            group_decay_rate=trial.suggest_float("group_decay_rate", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.1),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-24 16:10:49,847] A new study created in memory with name: no-name-b404063c-a6ea-445a-987c-39d8846d2c55
[I 2023-04-24 16:10:49,852] Trial 0 finished with value: 0.057439564148598785 and parameters: {'group_num': 8, 'within_decay_rate': 0.1, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 375, 'alpha': 0.5}. Best is trial 0 with value: 0.057439564148598785.
[I 2023-04-24 16:10:49,857] Trial 1 finished with value: 0.013697305773809882 and parameters: {'group_num': 4, 'within_decay_rate': 0.2, 'group_decay_rate': 0.8, 'nearest_neighbors_num': 100, 'alpha': 0.0}. Best is trial 0 with value: 0.057439564148598785.
[I 2023-04-24 16:10:49,862] Trial 2 finished with value: 0.06671495591256227 and parameters: {'group_num': 8, 'within_decay_rate': 1.0, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 275, 'alpha': 0.8}. Best is trial 2 with value: 0.06671495591256227.
[I 2023-04-24 16:10:49,866] Trial 3 finished with value: 0.0691401981306852 and parameters: {'group_nu

TIFUKNN fitting...


100%|██████████| 9307/9307 [30:48<00:00,  5.03it/s]
[I 2023-04-24 16:42:21,613] Trial 15 finished with value: 0.06557456152146485 and parameters: {'group_num': 3, 'within_decay_rate': 1.0, 'group_decay_rate': 0.30000000000000004, 'nearest_neighbors_num': 400, 'alpha': 0.8}. Best is trial 10 with value: 0.0702955950429197.


TIFUKNN fitting...


100%|██████████| 9307/9307 [33:45<00:00,  4.60it/s]
[I 2023-04-24 17:16:51,022] Trial 16 finished with value: 0.07006161576267411 and parameters: {'group_num': 2, 'within_decay_rate': 0.7000000000000001, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 500, 'alpha': 1.0}. Best is trial 10 with value: 0.0702955950429197.


...

100%|██████████| 9307/9307 [33:21<00:00,  4.65it/s]
[I 2023-04-24 20:28:05,436] Trial 22 finished with value: 0.06155363566136018 and parameters: {'group_num': 5, 'within_decay_rate': 0.7000000000000001, 'group_decay_rate': 0.1, 'nearest_neighbors_num': 500, 'alpha': 0.9}. Best is trial 18 with value: 0.07048305004009849.


TIFUKNN fitting...


100%|██████████| 9307/9307 [31:07<00:00,  4.98it/s]
[I 2023-04-24 20:59:56,908] Trial 23 finished with value: 0.07366860344168423 and parameters: {'group_num': 10, 'within_decay_rate': 0.6, 'group_decay_rate': 0.2, 'nearest_neighbors_num': 400, 'alpha': 1.0}. Best is trial 23 with value: 0.07366860344168423.


TIFUKNN fitting...


100%|██████████| 9307/9307 [31:15<00:00,  4.96it/s]
[I 2023-04-24 21:31:55,106] Trial 24 finished with value: 0.07059738999058329 and parameters: {'group_num': 10, 'within_decay_rate': 0.6, 'group_decay_rate': 0.30000000000000004, 'nearest_neighbors_num': 400, 'alpha': 0.9}. Best is trial 23 with value: 0.07366860344168423.


Test:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": TIFUKNN(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        group_num=study.best_params["group_num"],
        within_decay_rate=study.best_params["within_decay_rate"],
        group_decay_rate=study.best_params["group_decay_rate"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 10092/10092 [00:41<00:00, 241.42it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 30982.39it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 22587.21it/s]

TIFUKNN fitting...



100%|██████████| 10092/10092 [00:15<00:00, 638.46it/s]


In [ ]:
trainer.evaluate(mode="test")

100%|██████████| 9307/9307 [24:05<00:00,  6.44it/s]


{'precision': 0.0076823895992263885,
 'recall': 0.07491672934350488,
 'ndcg': 0.052395387393749784}

# Dunnhumby

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 2500, #items = 92339, #clicks = 2595370 (#illegal records = 0)
After preprocessing: #users = 2358, #items = 26756, #clicks = 1976796
Saving dataset in ./data//data_dunnhumby/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:08<00:00, 271.42it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:10<00:00, 222.97it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:09<00:00, 235.88it/s]


In [ ]:
def objective(trial):
    params = {
        "model": TIFUKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            group_num=trial.suggest_int("group_num", low=2, high=10, step=1),
            within_decay_rate=trial.suggest_float("within_decay_rate", 0.1, 1.0, step=0.1),
            group_decay_rate=trial.suggest_float("group_decay_rate", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.1),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-24 12:33:18,099] A new study created in memory with name: no-name-96ff447b-41a0-4de7-9e69-adf3145aa2f1


TIFUKNN fitting...


100%|██████████| 2357/2357 [05:29<00:00,  7.15it/s]
[I 2023-04-24 12:41:27,001] Trial 0 finished with value: 0.09253603345186663 and parameters: {'group_num': 8, 'within_decay_rate': 0.1, 'group_decay_rate': 0.7000000000000001, 'nearest_neighbors_num': 375, 'alpha': 0.5}. Best is trial 0 with value: 0.09253603345186663.


TIFUKNN fitting...


100%|██████████| 2357/2357 [02:59<00:00, 13.17it/s]
[I 2023-04-24 12:46:28,086] Trial 1 finished with value: 0.07525198420938144 and parameters: {'group_num': 4, 'within_decay_rate': 0.2, 'group_decay_rate': 0.8, 'nearest_neighbors_num': 100, 'alpha': 0.0}. Best is trial 0 with value: 0.09253603345186663.


...

100%|██████████| 2357/2357 [02:58<00:00, 13.19it/s]
[I 2023-04-24 14:52:06,481] Trial 22 finished with value: 0.1599942690657107 and parameters: {'group_num': 5, 'within_decay_rate': 0.9, 'group_decay_rate': 0.4, 'nearest_neighbors_num': 125, 'alpha': 0.30000000000000004}. Best is trial 15 with value: 0.17068007003990598.


TIFUKNN fitting...


100%|██████████| 2357/2357 [02:44<00:00, 14.32it/s]
[I 2023-04-24 14:56:47,045] Trial 23 finished with value: 0.1352820656812531 and parameters: {'group_num': 3, 'within_decay_rate': 0.7000000000000001, 'group_decay_rate': 0.6, 'nearest_neighbors_num': 50, 'alpha': 0.5}. Best is trial 15 with value: 0.17068007003990598.


TIFUKNN fitting...


100%|██████████| 2357/2357 [03:00<00:00, 13.05it/s]
[I 2023-04-24 15:01:50,118] Trial 24 finished with value: 0.1575971068680349 and parameters: {'group_num': 4, 'within_decay_rate': 0.9, 'group_decay_rate': 0.2, 'nearest_neighbors_num': 150, 'alpha': 0.9}. Best is trial 15 with value: 0.17068007003990598.


Test:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": TIFUKNN(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        group_num=study.best_params["group_num"],
        within_decay_rate=study.best_params["within_decay_rate"],
        group_decay_rate=study.best_params["group_decay_rate"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 2358/2358 [00:09<00:00, 243.37it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:08<00:00, 279.67it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:09<00:00, 240.19it/s]


TIFUKNN fitting...


100%|██████████| 2358/2358 [00:43<00:00, 54.73it/s]


In [ ]:
trainer.evaluate(mode="test")

100%|██████████| 2357/2357 [01:52<00:00, 20.92it/s]


{'precision': 0.11565549427238016,
 'recall': 0.16531576301767698,
 'ndcg': 0.16128563456518297}